In [0]:
%tensorflow_version 1.x

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import gc, sys
from tqdm import tqdm_notebook as tqdm

tf.__version__

'1.15.0'

In [0]:
#Set to true if computing on TPU
TPU = True

NB_EPOCHS = 50

#TPU 128, GPU : 32
BATCH_SIZE = 256

##Download data from Kaggle
1. upload the kaggle.json file before running the next cells
2. execute next cell

In [4]:
!cp ./kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

import kaggle as kg

api = kg.KaggleApi('kaggle.json')
api.authenticate()
api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions', './',
                           quiet=False, unzip=True)

food-com-recipes-and-user-interactions.zip: Skipping, found more recently modified local copy (use --force to force download)


## Build model Data input from PP_recipes.csv

In [5]:
#read the recipes file
pp_recipes = pd.read_csv('PP_recipes.csv')
pp_recipes.sort_values(['i'], inplace=True)
pp_recipes.head(3)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
46053,40893,0,"[40480, 1454, 16201, 2056, 955, 541, 11332, 82...","[[1424, 8876, 11007], [3484, 21453], [38966, 2...","[40480, 40482, 23667, 11007, 240, 21453, 240, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...",0,"[3384, 7979, 2127, 3502, 3217, 1257, 2778, 500..."
108526,44394,1,"[40480, 34712, 22683, 11274, 5409, 29868, 40481]","[[5343, 535, 2044, 5409, 7087], [17869, 6020],...","[40480, 40482, 12172, 1281, 5409, 7087, 240, 6...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[912, 7557, 2499, 5382]"
41248,85009,2,"[40480, 12187, 13995, 571, 14719, 40481]","[[1061, 494, 813, 2141], [31843], [30645, 4785...","[40480, 40482, 2572, 19472, 31757, 512, 823, 4...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...",2,"[4623, 6265, 1168, 6016, 3597, 3440, 7213, 169..."


In [6]:
# we need to create a collection of ingredients_ids to build vectors of recipes
set_ingredients = set()

for i in tqdm(range(len(pp_recipes['ingredient_ids']))):
    current = [int(x.strip()) for x in pp_recipes['ingredient_ids'].values[i][1:-1].split(',')]
    set_ingredients.update(current)
    
print('Number of ingredients in total :', len(set_ingredients))


Number of ingredients in total : 7993


In [7]:
list_ingredients = list(set_ingredients)

#initialize the matrix
np_matrix = np.zeros((len(pp_recipes), list_ingredients[-1] + 1),dtype=np.uint8)

#populate it
for i in tqdm(range(len(pp_recipes['ingredient_ids']))):
    for x in pp_recipes['ingredient_ids'].values[i][1:-1].split(','):
        np_matrix[i,int(x.strip())] = 1

In [8]:
#wrap the matrix in a pandas dataframe
pd_matrix = pd.DataFrame(np_matrix,columns=range(list_ingredients[-1] + 1), index=pp_recipes['i'].values,dtype=np.uint8)

del np_matrix
gc.collect()

pd_matrix.shape

(178265, 8023)

In [9]:
#Computing values for tensorflow
STEPS_PER_EPOCH = len(pd_matrix) // BATCH_SIZE

print('Epochs :', NB_EPOCHS,',Batch size :', BATCH_SIZE, ',Step per epochs :', STEPS_PER_EPOCH)

Epochs : 50 ,Batch size : 256 ,Step per epochs : 696


In [0]:
#build an input dataSet using tf.data.Dataset tensorflow api (looks like spark)
def BuildTFDataset(np_matrix):
  data = tf.data.Dataset.from_tensor_slices(np_matrix)

  #TPU can't deal with uint8, cast to int32
  # (X, (X', dummy))  
  data = data.map(lambda x : (tf.cast(x, tf.int32), (tf.cast(x, tf.int32), 0)))
  data = data.repeat(NB_EPOCHS)
  data = data.shuffle(2 * BATCH_SIZE * STEPS_PER_EPOCH)
  data = data.batch(BATCH_SIZE, drop_remainder=TPU)

  return data

In [0]:
#to debug dataset if needed

#with tf.Session() as sess:
#  test = sess.run(BuildTFDataset(pd_matrix).make_one_shot_iterator().get_next())
#  print(test[0].shape)

#Build the model and start training

In [0]:
def BuildAEModel(n_ingredients, activation=None):
    inputs = tf.keras.layers.Input((n_ingredients,))
    #encoded_layer1 = tf.keras.layers.Dense(8192,activation=None, name='Encoder_Layer_1')(inputs)
    #encoded_layer2 = tf.keras.layers.Dense(4096,activation=None, name='Encoder_Layer_2')(encoded_layer1)
    #encoded_layer3 = tf.keras.layers.Dense(2048,activation=None, name='Encoder_Layer_3')(encoded_layer2)
    
    embedded = tf.keras.layers.Dense(1024,activation=activation, name='embedder')(inputs)
    
    #decoded_layer1 = tf.keras.layers.Dense(2048,activation=None, name='Decoder_Layer_1')(embedded)
    #decoded_layer2 = tf.keras.layers.Dense(4096,activation=None, name='Decoder_Layer_2')(decoded_layer1)
    #decoded_layer3 = tf.keras.layers.Dense(8192,activation=None, name='Decoder_Layer_3')(decoded_layer2)
    
    outputs = tf.keras.layers.Dense(n_ingredients, activation='sigmoid', name = 'Reconstructor')(embedded)
    
    model = tf.keras.Model(inputs=inputs, outputs = [outputs, embedded])
    return model

In [13]:
#initialize tpu only once
if TPU and not('strategy' in globals()):
  resolver = tf.contrib.cluster_resolver.TPUClusterResolver()
  tf.contrib.distribute.initialize_tpu_system(resolver)
  strategy = tf.contrib.distribute.TPUStrategy(resolver)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Initializing the TPU system: 10.87.114.194:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.87.114.194:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 4700023981800189271)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1404438085793916144)
I

In [14]:
if TPU :
  with strategy.scope():
    print("TPU Mode activated")
    my_model = BuildAEModel(pd_matrix.shape[1], 'relu')
    my_model.summary()
    my_model.compile('adam',loss=['mse', lambda y_true, y_preds : 0.0], metrics={'Reconstructor':tf.keras.metrics.binary_accuracy})
else:
    my_model = BuildAEModel(pd_matrix.shape[1], 'relu')
    my_model.summary()
    my_model.compile('adam',loss=['mse', lambda y_true, y_preds : 0.0], metrics={'Reconstructor':tf.keras.metrics.binary_accuracy})

TPU Mode activated
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8023)]            0         
_________________________________________________________________
embedder (Dense)             (None, 1024)              8216576   
_________________________________________________________________
Reconstructor (Dense)        (None, 8023)              8223575   
Total params: 16,440,151
Trainable params: 16,440,151
Non-trainable params: 0
_________________________________________________________________


In [0]:
# This function decrease the learning rate by 10 at each epoch
def scheduler(epoch):
    return 0.001 * np.power(0.1, epoch)

callbacks = [tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)]
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',patience=1, verbose=1), 
             tf.keras.callbacks.EarlyStopping(monitor = 'loss',patience = 3)]


In [16]:
hist = my_model.fit(BuildTFDataset(pd_matrix.values), epochs=NB_EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, callbacks=callbacks)

Epoch 1/50
696/696 [==============================] - 48s 69ms/step - loss: 0.0073 - Reconstructor_loss: 0.0073 - embedder_loss: 0.0000e+00 - Reconstructor_binary_accuracy: 0.9969
Epoch 2/50
696/696 [==============================] - 18s 26ms/step - loss: 8.7722e-04 - Reconstructor_loss: 8.7722e-04 - embedder_loss: 0.0000e+00 - Reconstructor_binary_accuracy: 0.9991
Epoch 3/50
696/696 [==============================] - 18s 26ms/step - loss: 7.6652e-04 - Reconstructor_loss: 7.6652e-04 - embedder_loss: 0.0000e+00 - Reconstructor_binary_accuracy: 0.9992
Epoch 4/50
695/696 [============================>.] - ETA: 0s - loss: 6.9427e-04 - Reconstructor_loss: 6.9427e-04 - embedder_loss: 0.0000e+00 - Reconstructor_binary_accuracy: 0.9993
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
696/696 [==============================] - 18s 27ms/step - loss: 6.9429e-04 - Reconstructor_loss: 6.9429e-04 - embedder_loss: 0.0000e+00 - Reconstructor_binary_accuracy: 0.9993
Epoc

In [17]:
#execute the evaluate method to have better metrics
NB_EPOCHS=1 #ensure that the Dataset will be built only with on
my_model.evaluate(BuildTFDataset(pd_matrix.values), verbose=1)

696/696 [==============================] - 22s 32ms/step


[0.0006770939908583177, 0.0006770939, 0.0, 0.9993267]

In [0]:
res = my_model.predict(pd_matrix.values[0,:].reshape(1, -1).astype(np.int32))

In [19]:
print('number of ingredients for recipe 0 :', pd_matrix.values[0,:].sum())
print('number of ingredients for recipe 0 (reconstruction) :', res[0][0].sum())

number of ingredients for recipe 0 : 9
number of ingredients for recipe 0 (reconstruction) : 6.2339625


In [20]:
res[0].shape

(8, 8023)

In [21]:
res[1].shape

(8, 1024)

In [22]:
pd_matrix.values.sum(axis=1).mean()

8.991686534092503